# Data Preprocessing of Presidential Debate 2024 (Trump vs Harris)

In [ ]:
# load the abcnews_transcript file from the data folder

with open('data/abcnews_transcripts.csv', 'r') as file:
    data = file.read()

# print the first 1000 characters of the file
print(data[:1000])